<a href="https://colab.research.google.com/github/mdapoy/Machine-Learning-week-8-16/blob/main/Ch19_TRAINING_AND_DEPLOYING_TENSORFLOW_MODELS_AT_SCALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAPORAN ANALISIS BUKU
**Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow (Aurélien Géron)**

**BAB 19: TRAINING AND DEPLOYING TENSORFLOW MODELS AT SCALE**

## 1. Pendahuluan
Bab 19 membahas langkah-langkah penting setelah model *Machine Learning* dilatih: penyebaran (*deployment*) dan pelatihan berskala besar. Model yang telah dilatih perlu ditempatkan di lingkungan produksi untuk digunakan secara langsung. Seringkali, ini melibatkan penyediaan model sebagai layanan web yang dapat diakses melalui API (seperti REST atau gRPC). [cite_start]Pembaruan model secara berkala dengan data baru, pengelolaan versi model, transisi yang mulus antar versi, dan kemampuan untuk melakukan eksperimen A/B adalah aspek krusial dari *deployment* ini. [cite_start]Jika produk berhasil, layanan prediksi harus mampu menskalakan untuk mendukung volume *queries per second* (QPS) yang tinggi.

Pelatihan model berskala besar juga menjadi tantangan, terutama dengan volume data yang masif dan model yang intensif komputasi. Waktu pelatihan yang lama dapat menghambat eksperimen ide baru dan adaptasi cepat terhadap perubahan data. [cite_start]Solusinya mencakup penggunaan *hardware accelerator* seperti GPU atau TPU, dan mendistribusikan komputasi ke banyak mesin, masing-masing dilengkapi dengan banyak *accelerator*. [cite_start]TensorFlow menyediakan *Distribution Strategies API* untuk mempermudah hal ini.

## 2. Menyediakan Model TensorFlow (Serving a TensorFlow Model)
Setelah model TensorFlow dilatih, penggunaannya dalam kode Python cukup mudah (misalnya, memanggil metode `predict()` pada model `tf.keras`). [cite_start]Namun, seiring berkembangnya infrastruktur, lebih baik menyediakan model sebagai layanan terpisah yang hanya bertugas membuat prediksi.

* **Keuntungan Menggunakan Layanan Model**:
    * **Dekopling**: Model terpisah dari infrastruktur lain.
    * **Pembaruan Mudah**: Memungkinkan transisi dan pembaruan versi model yang mudah.
    * **Skalabilitas**: Layanan dapat diskalakan secara independen untuk menangani beban tinggi.
    * **Eksperimen A/B**: Memungkinkan pengujian beberapa versi model secara paralel.
    * **Konsistensi**: Memastikan semua komponen perangkat lunak bergantung pada versi model yang sama.
    * [cite_start]**Pengujian dan Pengembangan yang Lebih Sederhana**.

### 2.1. Menggunakan TensorFlow Serving (TF Serving)
[cite_start]TF Serving adalah *model server* yang sangat efisien dan teruji yang ditulis dalam C++.

* **Fitur Utama**:
    * Mampu menangani beban tinggi.
    * Melayani beberapa versi model secara bersamaan.
    * Memantau repositori model untuk secara otomatis menyebarkan versi terbaru.
    * [cite_start]Mendukung API gRPC (port 8500 secara default) dan REST (port 8501 secara default).
    * [cite_start]Secara otomatis melakukan *batching* permintaan untuk meningkatkan kinerja pada GPU.

#### 2.1.1. Mengekspor SavedModels
[cite_start]Model TensorFlow diekspor ke format *SavedModel* menggunakan `tf.saved_model.save()`. [cite_start]Ini menyimpan grafik komputasi model dan bobotnya.

* **Pentingnya Pra-pemrosesan dalam Model**: Sebaiknya sertakan semua lapisan pra-pemrosesan dalam model yang diekspor agar dapat menerima data dalam bentuk mentahnya di produksi. [cite_start]Ini menyederhanakan pembaruan dan mengurangi risiko ketidakcocokan antara model dan langkah pra-pemrosesan yang diperlukan.
* [cite_start]**Keterbatasan SavedModel**: Hanya dapat digunakan dengan model yang secara eksklusif berbasis operasi TensorFlow, tidak termasuk kode Python (`tf.py_function()`) atau model `tf.keras` dinamis.
* [cite_start]**Struktur Direktori SavedModel**: Terdiri dari direktori versi (misalnya, `0001`), yang berisi `saved_model.pb` (grafik komputasi sebagai *protocol buffer* serial), dan subdirektori `variables` (nilai variabel model).
* [cite_start]**Inspeksi SavedModel**: Alat baris perintah `saved_model_cli` dapat digunakan untuk memeriksa *SavedModel* dan detail tanda tangannya (input/output).

#### 2.1.2. Menginstal TensorFlow Serving
[cite_start]Instalasi TF Serving dapat dilakukan melalui citra Docker, manajer paket sistem, atau dari sumber. [cite_start]Docker sangat direkomendasikan karena kemudahan instalasi, isolasi sistem, dan kinerja tinggi.

#### 2.1.3. Mengirim Kueri TF Serving melalui REST API
Permintaan ke TF Serving melalui REST API harus dalam format JSON, termasuk nama tanda tangan fungsi dan data input. [cite_start]Meskipun sederhana dan banyak didukung klien, ini kurang efisien untuk data besar karena format berbasis teks dan verbositasnya.

#### 2.1.4. Mengirim Kueri TF Serving melalui gRPC API
gRPC API lebih efisien karena menggunakan *protocol buffer* serial sebagai input dan output, yang merupakan format biner yang ringkas. [cite_start]Ini jauh lebih baik untuk transfer data dalam jumlah besar.

#### 2.1.5. Menyebarkan Versi Model Baru
TF Serving secara berkala memeriksa versi model baru. Ketika versi baru terdeteksi, ia menangani transisi secara mulus: permintaan yang tertunda dijawab dengan versi lama, sementara permintaan baru dilayani oleh versi baru. Setelah semua permintaan tertunda selesai, versi lama dibongkar. [cite_start]Ini memastikan transisi yang lancar.

* [cite_start]**Model Warmup**: TF Serving dapat dikonfigurasi untuk mengeksekusi model pada *example instances* sebelum melayani permintaan baru, memastikan semuanya dimuat dengan benar dan menghindari waktu respons yang lama untuk permintaan pertama.
* [cite_start]**Skalabilitas**: Untuk menangani QPS yang tinggi, TF Serving dapat disebarkan di beberapa server dengan *load balancing*. [cite_start]Kubernetes adalah alat orkestrasi kontainer yang dapat digunakan untuk mengelola banyak kontainer TF Serving di banyak server.

## 3. Membuat Layanan Prediksi di Google Cloud AI Platform (GCP AI Platform)
GCP AI Platform adalah layanan *cloud* yang mengelola *deployment* model TensorFlow, menawarkan fitur seperti skalabilitas otomatis, pemantauan, dan penggunaan TPU.

* **Langkah-langkah Penyiapan**:
    1.  [cite_start]Masuk ke konsol GCP dan terima persyaratan layanan.
    2.  [cite_start]Verifikasi atau aktifkan akun penagihan.
    3.  [cite_start]Buat atau pilih proyek GCP.
    4.  [cite_start]Buat *bucket* di Google Cloud Storage (GCS) untuk menyimpan *SavedModels* dan data.
    5.  [cite_start]Unggah folder *SavedModel* ke *bucket* GCS.
    6.  [cite_start]Konfigurasi AI Platform: buat model baru dan versinya, tentukan jalur model GCS, versi *runtime* TensorFlow, tipe mesin, dan penskalaan (otomatis direkomendasikan).
* [cite_start]**Skalabilitas Otomatis**: AI Platform akan meluncurkan lebih banyak kontainer TF Serving saat QPS meningkat, dan menghentikannya saat QPS menurun, mengaitkan biaya langsung dengan penggunaan.
* [cite_start]**Autentikasi**: Autentikasi klien untuk layanan prediksi harus menggunakan *service account* dengan hak akses yang sangat terbatas (misalnya, peran *ML Engine Developer*) untuk keamanan. [cite_start]Kunci pribadi *service account* harus dijaga kerahasiaannya.

## 4. Menyebarkan Model ke Perangkat Seluler atau Tertanam (Mobile or Embedded Device)
Untuk menyebarkan model ke perangkat seluler atau tertanam, model harus ringan dan efisien untuk menghindari masalah kinerja. [cite_start]Pustaka TFLite menyediakan alat untuk tujuan ini.

* **Tujuan TFLite**:
    * Mengurangi ukuran model.
    * Mengurangi jumlah komputasi per prediksi (mengurangi latensi, konsumsi baterai, dan panas).
    * Mengadaptasi model ke batasan perangkat spesifik.
* **Konversi Model**: *Converter* TFLite dapat mengompres *SavedModel* ke format yang lebih ringan berdasarkan *FlatBuffers*. [cite_start]Ini mengurangi waktu *loading* dan jejak memori karena *FlatBuffers* dapat dimuat langsung ke RAM.
* [cite_start]**Optimasi Model**: *Converter* juga mengoptimalkan model dengan memangkas operasi yang tidak perlu dan mengoptimalkan serta menggabungkan komputasi (misalnya, lapisan *Batch Normalization* dilipat ke dalam operasi lapisan sebelumnya).
* [cite_start]**Kuantisasi**: Mengurangi ukuran model lebih lanjut dengan mengkuantisasi bobot model ke bilangan bulat 8-bit *fixed-point*, menghasilkan pengurangan ukuran empat kali lipat dibandingkan *float* 32-bit.
    * **Post-training Quantization**: Mengkuantisasi bobot setelah pelatihan, menggunakan teknik kuantisasi simetris dasar. [cite_start]Namun, saat *runtime*, bobot dikonversi kembali ke *float*.
    * **Kuantisasi Penuh**: Mengkuantisasi bobot dan aktivasi sehingga komputasi dapat dilakukan sepenuhnya dengan bilangan bulat, mengurangi latensi dan konsumsi daya. [cite_start]Ini memerlukan langkah kalibrasi untuk menemukan nilai absolut maksimum aktivasi.
    * [cite_start]**Quantization-aware Training**: Menambahkan operasi kuantisasi palsu ke model selama pelatihan agar model belajar mengabaikan *noise* kuantisasi, membuat bobot akhir lebih kuat terhadap kuantisasi.

## 5. TensorFlow di Browser
[cite_start]Model dapat dijalankan langsung di *browser* pengguna menggunakan pustaka TensorFlow.js JavaScript.

* **Manfaat**:
    * **Konektivitas Terputus**: Berguna saat konektivitas pengguna terputus-putus atau lambat.
    * **Latensi Rendah**: Prediksi secepat mungkin karena tidak perlu kueri server.
    * [cite_start]**Privasi Pengguna**: Prediksi dilakukan di sisi klien, menjaga data pribadi pengguna tetap lokal.
* [cite_start]**tensorflowjs_converter**: Alat untuk mengonversi *SavedModel* atau file model Keras ke format TensorFlow.js Layers, yang dioptimalkan untuk pengunduhan web.

## 6. Menggunakan GPU untuk Mempercepat Komputasi
[cite_start]Pelatihan jaringan saraf besar pada satu CPU bisa memakan waktu berhari-hari atau berminggu-minggu. GPU dapat secara signifikan mempercepat pelatihan.

* [cite_start]**Mendapatkan GPU Sendiri**: Memerlukan pembelian kartu GPU Nvidia dengan CUDA Compute Capability 3.5+ dan instalasi *driver* Nvidia, CUDA, serta cuDNN (pustaka yang dipercepat GPU untuk DNNs).
* **Menggunakan Mesin Virtual yang Dilengkapi GPU**: Platform *cloud* besar menawarkan VM GPU yang sudah dikonfigurasi sebelumnya dengan semua *driver* dan pustaka yang diperlukan (termasuk TensorFlow). Google Cloud AI Platform menawarkan VM *Deep Learning* dan *Colaboratory* (Colab) untuk akses GPU gratis.
* **Mengelola RAM GPU**: TensorFlow secara default mengambil semua RAM GPU yang tersedia pada komputasi pertama untuk membatasi fragmentasi.
    * **Pembagian GPU antar Proses**: Gunakan variabel lingkungan `CUDA_VISIBLE_DEVICES` untuk menetapkan GPU tertentu ke proses individual.
    * [cite_start]**Membatasi Penggunaan RAM GPU**: Konfigurasi TensorFlow untuk mengambil hanya sejumlah RAM GPU tertentu menggunakan `tf.config.experimental.set_virtual_device_configuration`.
    * **Pertumbuhan Memori Sesuai Permintaan**: TensorFlow dapat mengambil memori hanya saat dibutuhkan menggunakan `tf.config.experimental.set_memory_growth`.
    * [cite_start]**Virtual GPU**: Membagi satu GPU fisik menjadi dua atau lebih GPU virtual.
* **Menempatkan Operasi dan Variabel pada Perangkat**: `tf.keras` dan `tf.data` biasanya menempatkan operasi dan variabel secara optimal. [cite_start]Namun, penempatan manual dapat dilakukan menggunakan konteks `tf.device()` untuk kontrol lebih lanjut (misalnya, pra-pemrosesan di CPU, komputasi NN di GPU).
* **Eksekusi Paralel Lintas Beberapa Perangkat**: Ketika TensorFlow menjalankan `TF Function`, ia menganalisis grafiknya untuk mengeksekusi operasi secara paralel pada perangkat yang berbeda. Operasi pada CPU dijalankan di *thread pool* `inter-op`, dan operasi yang memiliki kernel *multithreaded* menggunakan *thread pool* `intra-op`. Operasi GPU dijalankan secara berurutan, tetapi kernel GPU yang *multithreaded* memanfaatkan banyak *thread* GPU secara paralel.

## 7. Melatih Model Lintas Beberapa Perangkat (Training Models Across Multiple Devices)
Ada dua pendekatan utama untuk melatih satu model di beberapa perangkat:

### 7.1. Model Parallelism
* **Konsep**: Membagi model menjadi bagian-bagian terpisah dan menjalankan setiap bagian pada perangkat yang berbeda.
* **Keterbatasan**: Sulit diterapkan secara efisien untuk jaringan yang *fully connected* karena tingginya komunikasi lintas perangkat. Lebih cocok untuk arsitektur dengan koneksi parsial seperti CNN.

### 7.2. Data Parallelism
* **Konsep**: Mereplikasi model pada setiap perangkat dan menjalankan setiap langkah pelatihan secara bersamaan pada semua replika, menggunakan *mini-batch* yang berbeda untuk setiap replika. [cite_start]Gradien yang dihitung oleh setiap replika kemudian dirata-ratakan dan hasilnya digunakan untuk memperbarui parameter model.

#### 7.2.1. Data Parallelism Menggunakan Mirrored Strategy
* **Konsep**: Semua parameter model dicerminkan sepenuhnya di semua GPU, dan pembaruan parameter yang sama persis selalu diterapkan pada setiap GPU. Semua replika tetap identik sempurna.
* [cite_start]**Efisiensi**: Sangat efisien, terutama pada satu mesin.
* **Mekanisme**: Menggunakan algoritma *AllReduce* (misalnya, NVIDIA Collective Communications Library - NCCL) untuk menghitung rata-rata gradien secara efisien dan mendistribusikan hasilnya ke semua GPU.
* [cite_start]**Implementasi Keras**: Gunakan `tf.distribute.MirroredStrategy()` dan bungkus pembuatan serta kompilasi model di dalam *scope* strategi ini.

#### 7.2.2. Data Parallelism dengan Parameter Terpusat
* [cite_start]**Konsep**: Parameter model disimpan di luar perangkat GPU yang melakukan komputasi (disebut *workers*), misalnya pada CPU atau server parameter khusus.

* **Pembaruan Sinkron**: *Aggregator* menunggu hingga semua gradien tersedia sebelum menghitung rata-rata gradien dan meneruskannya ke *optimizer*. Semua perangkat harus menunggu yang paling lambat.
* **Pembaruan Asinkron**: Setiap replika memperbarui parameter model segera setelah selesai menghitung gradien. Tidak ada agregasi atau sinkronisasi. Replika bekerja secara independen. [cite_start]Ini dapat menyebabkan *stale gradients* (gradien yang sudah ketinggalan zaman) yang dapat memperlambat konvergensi atau menyebabkan divergensi.

* **Saturasi Bandwidth**: Baik *mirrored strategy* maupun parameter terpusat melibatkan komunikasi data yang signifikan. [cite_start]Ada titik di mana menambahkan lebih banyak GPU tidak lagi meningkatkan kinerja karena waktu yang dihabiskan untuk memindahkan data masuk dan keluar dari RAM GPU (dan melalui jaringan dalam pengaturan terdistribusi) melebihi peningkatan kecepatan komputasi.
    * Lebih parah untuk model *dense* besar.
    * [cite_start]Kurang parah untuk model *sparse* besar, di mana gradien sebagian besar nol dan dapat dikomunikasikan secara efisien.
    * [cite_start]Solusi: Menggunakan beberapa GPU yang kuat daripada banyak GPU lemah, mengelompokkan GPU pada server yang saling terhubung dengan baik, dan mengurangi presisi *float* (misalnya, dari 32 bit ke 16 bit).

### 7.3. Melatih pada Skala Menggunakan Distribution Strategies API
TensorFlow menyediakan *Distribution Strategies API* untuk mempermudah pelatihan model berskala besar.

* **MultiWorkerMirroredStrategy**: Mirrored data parallelism di banyak server. Model direplikasi di semua server dan perangkat, dan setiap replika mendapatkan *batch* data yang berbeda. Rata-rata gradien dihitung dan dibagikan menggunakan implementasi *AllReduce* terdistribusi (NCCL secara default). Ini adalah strategi yang direkomendasikan secara umum.
* **ParameterServerStrategy**: Asinkron data parallelism dengan *parameter server*. [cite_start]Model direplikasi di semua perangkat pada semua *worker*, dan parameter dipecah di seluruh *parameter server*.
* **TPUStrategy**: Untuk menggunakan TPU di Google Cloud.

## 8. Menjalankan Pekerjaan Pelatihan Besar di Google Cloud AI Platform
*Google Cloud AI Platform* memungkinkan *deployment* pekerjaan pelatihan menggunakan kode pelatihan yang sama seperti pada *cluster* TF Anda sendiri, sambil mengelola penyediaan dan konfigurasi VM GPU.

* [cite_start]**gcloud command-line tool**: Digunakan untuk memulai pekerjaan pelatihan, menentukan wilayah, *scale tier* (misalnya, `PREMIUM_1` untuk banyak *workers* dan *parameter servers*), versi *runtime*, jalur kode, dll.
* **Google Cloud Storage (GCS)**: Digunakan untuk data pelatihan.
* [cite_start]**Pemantauan**: Konsol GCP menyediakan grafik pemanfaatan CPU, GPU, dan RAM, serta *log* detail menggunakan *Stackdriver*.

### 8.1. Penyetelan Hiperparameter Black Box di AI Platform
[cite_start]AI Platform menyediakan layanan penyetelan hiperparameter *Bayesian optimization* yang kuat bernama *Google Vizier*.

* [cite_start]**Konfigurasi**: Melalui file konfigurasi YAML, menentukan tujuan (misalnya, `MAXIMIZE`), metrik hiperparameter (misalnya, `accuracy`), jumlah *trial* maksimum, *trial* paralel maksimum, dan parameter yang akan disetel dengan rentang dan *scale type* (misalnya, `UNIT_LINEAR_SCALE`, `UNIT_LOG_SCALE`).
* **Komunikasi Metrik**: Kode pelatihan berkomunikasi metrik kembali ke AI Platform dengan menggunakan *callback* `TensorBoard()` (yang menulis *event file* yang dipantau oleh AI Platform).

---

## REPRODUCE CODE

## Exporting SavedModels

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

# Asumsikan model sudah dibuat dan dilatih
# model = keras.models.Sequential([...])
# model.compile([...])
# history = model.fit([...])

model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
# tf.saved_model.save(model, model_path) # Baris ini akan menyimpan model

Kode berikut menunjukkan cara memuat kembali SavedModel dan menggunakannya untuk prediksi, serta cara menggunakan saved_model_cli dari terminal.

In [2]:
# Memuat model yang tersimpan
# saved_model = tf.saved_model.load(model_path)
# y_pred = saved_model(tf.constant(X_new, dtype=tf.float32))

# Memuat model sebagai model Keras
# model = keras.models.load_model(model_path)
# y_pred = model.predict(tf.constant(X_new, dtype=tf.float32))

# Menyimpan data tes untuk digunakan dengan CLI
import numpy as np
# X_new = np.random.rand(10, 28, 28) # Contoh data baru
# np.save("my_mnist_tests.npy", X_new)

Perintah Terminal untuk saved_model_cli:

In [4]:
# Mengekspor path ke variabel lingkungan (opsional, untuk kenyamanan)
# export ML_PATH="$HOME/ml"
# cd $ML_PATH

# Menampilkan informasi SavedModel
# saved_model_cli show --dir my_mnist_model/0001 --all

# Menjalankan prediksi menggunakan CLI
# saved_model_cli run --dir my_mnist_model/0001 --tag_set serve \
#  --signature_def serving_default \
#  --inputs flatten_input=my_mnist_tests.npy

## Installing TensorFlow Serving

In [5]:
# Mengunduh image Docker resmi TF Serving
# docker pull tensorflow/serving

# Menjalankan container Docker untuk TF Serving
# docker run -it --rm -p 8500:8500 -p 8501:8501 \
#  -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
#  -e MODEL_NAME=my_mnist_model \
#  tensorflow/serving

## Querying TF Serving through the REST API

In [6]:
import json
import requests
import numpy as np

# Asumsikan X_new adalah data input (misalnya, dari data tes MNIST)
# X_new = [...]

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

y_proba = np.array(response["predictions"])
# print(y_proba.round(2))

## Querying TF Serving through the gRPC API

In [7]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis.predict_pb2 import PredictRequest

# Membuat permintaan gRPC
request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
# Asumsikan 'model' adalah model Keras yang sudah dimuat
# dan X_new adalah data input
# input_name = model.input_names[0]
# request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

# Mengirim permintaan
# channel = grpc.insecure_channel('localhost:8500')
# predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# response = predict_service.Predict(request, timeout=10.0)

# Mengonversi respons kembali ke tensor
# output_name = model.output_names[0]
# outputs_proto = response.outputs[output_name]
# y_proba = tf.make_ndarray(outputs_proto)

ModuleNotFoundError: No module named 'tensorflow_serving'

In [8]:
!pip install tensorflow-serving-api grpcio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow

## Deploying a new model version

In [ ]:
# Asumsikan model baru telah dilatih
# model = keras.models.Sequential([...])
# model.compile([...])
# history = model.fit([...])

model_version = "0002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
# tf.saved_model.save(model, model_path)

## Using the Prediction Service (on GCP)

In [9]:
import googleapiclient.discovery

# Mengatur environment variable untuk autentikasi
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

project_id = "your-gcp-project-id" # Ganti dengan project ID Anda
model_id = "my_mnist_model"
model_path = "projects/{}/models/{}".format(project_id, model_id)
ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

def predict(X):
    # Asumsikan 'output_name' sudah didefinisikan sebelumnya
    output_name = "dense_1" # Contoh dari buku, sesuaikan jika perlu

    input_data_json = {"signature_name": "serving_default",
                       "instances": X.tolist()}
    request = ml_resource.predict(name=model_path, body=input_data_json)
    response = request.execute()
    if "error" in response:
        raise RuntimeError(response["error"])
    return np.array([pred[output_name] for pred in response["predictions"]])

# Y_probas = predict(X_new)
# print(np.round(Y_probas, 2))

## Deploying a Model to a Mobile or Embedded Device

In [12]:
# Menyimpan model ke path
# saved_model_path = "my_mnist_model/0001"

# Mengonversi model ke TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()
with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)

# Menggunakan kuantisasi post-training
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model_quantized = converter.convert()

## Using GPUs to Speed Up Computations

In [11]:
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())
print(tf.config.experimental.list_physical_devices(device_type='GPU'))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True
/device:GPU:0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Mengatur pembatasan memori GPU (Memory Growth):

In [13]:
# Opsi 1: Mengatur batas memori virtual
# for gpu in tf.config.experimental.list_physical_devices("GPU"):
#     tf.config.experimental.set_virtual_device_configuration(
#         gpu,
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])

# Opsi 2: Mengizinkan memory growth
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

## Menempatkan operasi pada device tertentu:

In [14]:
with tf.device("/cpu:0"):
    c = tf.Variable(42.0)
print(c.device)

/job:localhost/replica:0/task:0/device:CPU:0


## Menggunakan MirroredStrategy (pada satu mesin dengan beberapa GPU):

In [ ]:
distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    mirrored_model = keras.models.Sequential([
        # ... lapisan-lapisan model Anda
    ])
    mirrored_model.compile(loss="...", optimizer="...")

batch_size = 100 # harus bisa dibagi dengan jumlah replika
# history = mirrored_model.fit(X_train, y_train, epochs=10)

# Memuat model dalam scope distribusi
# with distribution.scope():
#    mirrored_model = keras.models.load_model("my_mnist_model.h5")

## Menggunakan MultiWorkerMirroredStrategy (pada beberapa server):

In [ ]:
# Contoh TF_CONFIG untuk worker pertama
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",
        "machine-b.example.com:2222"
    ],
    "ps": ["machine-a.example.com:2221"] # Parameter Server
}

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 0}
})

# Kode training (dijalankan di setiap worker)
# distribution = tf.distribute.experimental.MultiWorkerMirroredStrategy()
# with distribution.scope():
#    mirrored_model = keras.models.Sequential([...])
#    mirrored_model.compile([...])
# batch_size = 100
# history = mirrored_model.fit(X_train, y_train, epochs=10)

## Running Large Training Jobs on Google Cloud AI Platform

In [ ]:
# gcloud ai-platform jobs submit training my_job_20190531_164700 \
#  --region asia-southeast1 \
#  --scale-tier PREMIUM_1 \
#  --runtime-version 2.0 \
#  --python-version 3.5 \
#  --package-path /my_project/src/trainer \
#  --module-name trainer.task \
#  --staging-bucket gs://my-staging-bucket \
#  --job-dir gs://my-mnist-model-bucket/trained_model \
#  -- \
#  --my-extra-argument1 foo --my-extra-argument2 bar